In [7]:
import astropy.units as u
import numpy as np
import naima
from scipy.optimize import fsolve

In [8]:
from gammapy.modeling.models import NaimaSpectralModel

In [9]:
import gammapy
gammapy.__version__

'0.15'

# How to normalize the NaimaSpectralModel using the differential flux at 1 TeV

The trick to normalize the model is using the function `fsolve` from scipy, that allows to find the value of the ampltitude for which the fux at 1 TeV has the desired value.

Here's the function:

In [12]:
def spectral_model_normalized(norm, index, cutoff):
    """Normalize the model via the differential flux at 1 TeV (in Crab units), 
     instead of using the model amplitude, as usually done in Naima."""
    
    crab_unit = "3.84e-11 cm-2 s-1 TeV-1" # Crab flux at 1 TeV
    
    def model_to_be_normalized(x):
        particle_distribution = naima.models.ExponentialCutoffPowerLaw(
            amplitude=x / u.eV, 
            e_0=10 * u.TeV, 
            alpha=index, 
            e_cutoff=cutoff * u.PeV
        )
        radiative_model = naima.radiative.PionDecay(
                particle_distribution,
                nh = 1 / u.cm**3,
        )
        return NaimaSpectralModel(radiative_model=radiative_model, distance=1*u.kpc)
        
    def f_pl(x):
        return (model_to_be_normalized(x)(1*u.TeV) / crab_unit).to("") - norm

    amplitude = fsolve(f_pl, 1e32)[0]
    np.testing.assert_almost_equal(f_pl(amplitude), 0)
    return model_to_be_normalized(amplitude)

Example:

In [14]:
%%time
model = spectral_model_normalized(0.05, 2.0, 1)

CPU times: user 1.18 s, sys: 28.5 ms, total: 1.21 s
Wall time: 1.21 s


let's check:

In [15]:
crab_unit = "3.84e-11 cm-2 s-1 TeV-1" # Crab flux at 1 TeV
(model(1 * u.TeV) / crab_unit).to("") 

<Quantity 0.05>